In [ ]:
from azureml.core import Experiment
from azureml.core import Workspace, Run
from azureml.core import Environment
from azureml.core import Dataset, Datastore

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig

In [ ]:
from azureml.core.compute import ComputeInstance

In [3]:
workspace = Workspace.from_config()

In [4]:
instance = ComputeTarget(workspace=workspace, name='gpu-compute-low')

In [5]:
dataset = Dataset.get_by_name(workspace, name='recursionbio')

In [6]:
dataset

{
  "source": [
    "('codefilestore', '/Users/kjaanson/recursionpharma/input/recbio/**')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "ee33756c-ab30-45ec-adf9-5b93629bbbd8",
    "name": "recursionbio",
    "version": 2,
    "workspace": "Workspace.create(name='bioworks-azureml', subscription_id='73f2271c-beee-4233-b5dd-202b520bf0db', resource_group='mslearn-azureml')"
  }
}

In [7]:
env_list = Environment.list(workspace)

In [8]:
env_list.keys()

dict_keys(['AzureML-VowpalWabbit-8.8.0', 'AzureML-PyTorch-1.3-CPU', 'AzureML-Minimal', 'AzureML-Tutorial', 'AzureML-PyTorch-1.5-GPU', 'AzureML-TensorFlow-2.2-CPU', 'AzureML-PyTorch-1.6-CPU', 'AzureML-PyTorch-1.5-CPU', 'AzureML-TensorFlow-2.2-GPU', 'AzureML-PyTorch-1.6-GPU', 'AzureML-TensorFlow-2.3-CPU', 'AzureML-Triton', 'AzureML-TensorFlow-2.3-GPU', 'AzureML-DeepSpeed-0.3-GPU', 'AzureML-Pytorch1.7-Cuda11-OpenMpi4.1.0-py36', 'AzureML-Scikit-learn0.24-Cuda11-OpenMpi4.1.0-py36', 'AzureML-TensorFlow2.4-Cuda11-OpenMpi4.1.0-py36', 'AzureML-TensorFlow-1.15-Inference-CPU', 'AzureML-XGBoost-0.9-Inference-CPU', 'AzureML-PyTorch-1.6-Inference-CPU', 'AzureML-Minimal-Inference-CPU'])

In [9]:
tf_env = Environment.get(workspace=workspace, name='AzureML-TensorFlow-2.3-GPU')
tf_env = tf_env.clone(new_name='recbio-tf-2.3-efficientnet')

In [10]:
tf_env.python.conda_dependencies.add_conda_package('scikit-learn')
tf_env.python.conda_dependencies.add_conda_package('scipy')
tf_env.python.conda_dependencies.add_conda_package('matplotlib')

In [11]:
tf_env.python.conda_dependencies.add_pip_package('horovod==0.19.5')
tf_env.python.conda_dependencies.add_pip_package('retry')

In [12]:
tf_env

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04:20210405.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "recbio-tf-2.3-efficientnet",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge"

In [13]:
train_scr = ScriptRunConfig(
    source_directory='./scripts',
    script='train_cnn.py',
    arguments=['--data-path', dataset.as_mount(),
               '--epochs', 10,
               '--batch', 32],
    compute_target=instance,
    environment=tf_env
)

In [14]:
train_scr

In [15]:
run = Experiment(workspace=workspace, name='recbio-effnet-model').submit(train_scr)

In [16]:
run.wait_for_completion(show_output=True)

RunId: recbio-effnet-model_1619355898_be089663
Web View: https://ml.azure.com/experiments/recbio-effnet-model/runs/recbio-effnet-model_1619355898_be089663?wsid=/subscriptions/73f2271c-beee-4233-b5dd-202b520bf0db/resourcegroups/mslearn-azureml/workspaces/bioworks-azureml

Streaming azureml-logs/65_job_prep-tvmps_8d3356cbc1241ec6d7c485396c9de4cf66674a75aefb56a54ddbdf3733489fa9_p.txt

[2021-04-25T13:09:09.850337] Entering job preparation.
[2021-04-25T13:09:10.546122] Starting job preparation.
[2021-04-25T13:09:10.546157] Extracting the control code.
[2021-04-25T13:09:10.564204] fetching and extracting the control code on master node.
[2021-04-25T13:09:10.564234] Starting extract_project.
[2021-04-25T13:09:10.564275] Starting to extract zip file.
[2021-04-25T13:09:10.988946] Finished extracting zip file.
[2021-04-25T13:09:11.111549] Using urllib.request Python 3.0 or later
[2021-04-25T13:09:11.111615] Start fetching snapshots.
[2021-04-25T13:09:11.111764] Start fetching snapshot.
[2021-04-

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code",
        "messageFormat": "{Message}",
        "messageParameters": {
            "Message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.\n\tReason: Job failed with non-zero exit Code"
        },
        "details": [],
        "innerError": {
            "code": "UserTrainingScriptFailed"
        }
    },
    "correlation": {
        "operation": null,
        "request": "8cd9f819ea77b44d"
    },
    "environment": "northeurope",
    "location": "northeurope",
    "time": "2021-04-25T13:16:42.367123Z",
    "componentName": "execution-worker"
}
	InnerException None
	ErrorResponse 
{
    "error"